## [Calculator I](https://leetcode.com/problems/basic-calculator/)
- Give a string of expression.
    + Contains: (, ), +, -
    + integer
- Evaluate the expression.

```
Input: "1 + 1"
Output: 2

Input: " 2-1 + 2 "
Output: 3

Input: "(1+(4+5+2)-3)+(6+8)"
Output: 23

Input: "59 + 19+(7-8-( 3+2)+3)"
Output: 75
```

#### Solution - Recursive

```C++
class Solution {
public:
    string s;
    int N;

    int calc(int &i) {
        int num = 0;
        int res = 0;
        int sign = 1; // '+':1, '-':-1

        while(i<N) {
            if(s[i] == '(') {
                ++i;
                res += sign*calc(i);
            }

            else if(s[i] == ')') {
                ++i;
                return res + sign*num;
            }

            else if(s[i] == '+') {
                res += sign*num;

                num = 0;
                sign = 1; 
                ++i;
            }

            else if(s[i] == '-') {
                res += sign*num;

                num = 0;
                sign = -1;
                ++i;
            }

            else if('0' <= s[i] && s[i] <= '9') {
                num = 10*num + (s[i]-'0');
                ++i;
            }

            else ++i;
        }

        return res + sign*num;
    }

    int calculate(const string &str) {
        s = str;
        N = s.size();

        int idx = 0;
        return calc(idx);
    }
};
```

#### Solution - Stack
- digit: Update current number
- '+': number is over
    + Add the previous number
    + start a new positive number
- '-': number is over
    + Add the previous number
    + start a new negative number
- '(': 
    + push the previous result and the sign into the stack,
    + set result to 0
    + calculate the new result within the parenthesis.
- ')': number is over
    + pop out the sign before this pair of parenthesis, and temporary result before this pair of parenthesis
    + Update result
    
```C++
int calculate(const string &s) {
    stack<int> S;
    int res = 0;
    int num = 0;
    int sign = 1;
    for(int i=0; i<s.size(); ++i) {
        // Case: digit 0 - 9
        if('0' <= s[i] && s[i] <= '9') {
            num = 10*num + (s[i]-'0');
        }

        // Case: +
        else if(s[i] == '+') {
            res += sign*num;

            num = 0;
            sign = 1;
        }

        // Case: -
        else if(s[i] == '-') {
            res += sign*num;

            num = 0;
            sign = -1;
        }

        // Case: (
        else if(s[i] == '(') {
            // push res, sign
            S.push(res);
            S.push(sign);

            res = 0;
            sign = 1;
        }

        // Case: )
        else if(s[i] == ')') {
            res += sign*num;
            num = 0;

            // pop sign, res
            res *= S.top(); S.pop();
            res += S.top(); S.pop();
        }
    }

    return res + sign*num;;
}
```

## [Calculator III](https://leetcode.com/problems/basic-calculator-iii/)
- Give a string of expression.
    + Contains: (, ), +, -, *, /
    + integer
- Evaluate the expression.

```
Input: "1 + 1"
Output: 2

Input: " 6-4 / 2 "
Output: 4

Input: "2*(5+5*2)/3+(6/2+8)"
Output: 21

Input: "(2+6* 3+5- (3*14/7+2)*5)+3"
Output: -12
```

#### Solution - Stack + Recursive
- digit: Update current number
- '+': number is over
    + Update */ result
    + Push stack result
- '-': number is over
    + Update */ result
    + Push stack result
- '*': number is over
    + Update */ result
- '/': number is over
    + Update */ result
- '(': 
    + treat () as a number
- ')':
    + return the solution in ()
    
```C++
class Solution {
public:
    typedef long long ll;
    string s;
    int N;

    ll calc(int &i) {
        ll num = 0;
        ll res = 0;
        ll term = 1;       // result of a*b/c

        ll sign = 1;       // '+':1, '-':-1
        ll mul_div = 1;    // '*':1, '/':0

        stack<ll> S;

        while(i<N) {
            if(s[i] == '(') {
                ++i;
                num = calc(i);
            }

            else if(s[i] == ')') {
                ++i;

                // Return result
                term = (mul_div == 1) ? term*num : term/num;
                S.push(sign*term);

                res = 0;
                while(!S.empty()) {
                    res += S.top();
                    S.pop();
                }
                return res;
            }

            else if(s[i] == '+') {
                term = (mul_div == 1) ? term*num : term/num;
                S.push(sign*term);

                num = 0;
                sign = 1;
                mul_div = 1;
                term = 1;
                ++i;
            }

            else if(s[i] == '-') {
                term = (mul_div == 1) ? term*num : term/num;
                S.push(sign*term);

                num = 0;
                sign = -1;
                mul_div = 1;
                term = 1;
                ++i;
            }

            else if(s[i] == '*') {
                term = (mul_div == 1) ? term*num : term/num;

                num = 0;
                mul_div = 1;
                ++i;
            }

            else if(s[i] == '/') {
                term = (mul_div == 1) ? term*num : term/num;

                num = 0;
                mul_div = 0;
                ++i;
            }

            else if('0' <= s[i] && s[i] <= '9') {
                num = 10*num + (s[i]-'0');
                ++i;
            }

            else ++i;
        }

        // Return result
        term = (mul_div == 1) ? term*num : term/num;
        S.push(sign*term);

        res = 0;
        while(!S.empty()) {
            res += S.top();
            S.pop();
        }
        return res;
    }

    int calculate(const string &str) {
        s = str;
        N = s.size();

        int idx = 0;
        return static_cast<int>(calc(idx));
    }
};
```